(working-with-data)=
# Working with Data


## Introduction

The previous chapter was just a quick tour of what can be done with a single tabular dataset (a 'dataframe'). In this chapter, we'll go deeper into working with data. [test]

The ability to extract, clean, and analyse data is one of the core skills any economist needs. Fortunately, the (open source) tools that are available for data analysis have improved enormously in recent years, and working with them can be a delight--even the most badly formatted data can be beaten into shape.

In this chapter, you'll get really good introduction to the [**pandas**](https://pandas.pydata.org/) package, the core data manipulation library in Python. The name is derived from 'panel data' but it's suited to any tabular data, and can be used to work with more complex data structures too. We *won't* cover reading in or writing data here; see the next chapter for that.

This chapter is hugely indebted to the fantastic [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/) by Jake Vanderplas. Remember, if you get stuck with pandas, there is brilliant [documentation](https://pandas.pydata.org/docs/user_guide/index.html) and a fantastic set of [introductory tutorials](https://pandas.pydata.org/pandas-docs/stable/getting_started/intro_tutorials/index.html) on their website. These notes are heavily indebted to those introductory tutorials.

This chapter uses the **pandas**, **seaborn**, and **numpy** packages. If you're running this code, you may need to install these packages, which you can do using either `conda install packagename` or `pip install packagename` on your computer's command line. (If you're not sure what a command line or terminal is, take a quick look at the basics of coding chapter.)


### Using tidy data

As an aside, if you're working with tabular data, it's good to try and use a so-called 'tidy data' format. This is data with one observation per row, and one variable per column, like so:

![](https://d33wubrfki0l68.cloudfront.net/6f1ddb544fc5c69a2478e444ab8112fb0eea23f8/91adc/images/tidy-1.png)

Tidy data aren't going to be appropriate *every* time and in every case, but they're a really, really good default for tabular data. Once you use it as your default, it's easier to think about how to perform subsequent operations. Some plotting libraries, such as **seaborn**, take that your data are in tidy format as a given. And many operations that you can perform on dataframes (the objects that hold tabular data within many programming languages) are easier when you have tidy data. If you're writing out data to file to share, putting it in tidy format is a really good idea.

Of course, *getting* your messy dataset into a tidy format may take a bit of work... but we're about to enter the exciting world of coding for data analysis and the tools you'll see in the rest of this chapter will help you to 'wrangle' even the most messy of datasets.

Having said that tidy data are great, and they are, one of standard data library **pandas**' advantages relative to other data analysis libraries is that it isn't *too* tied to tidy data and can navigate awkward non-tidy data manipulation tasks happily too.

## Dataframes and series

Let's start with the absolute basics. The most basic **pandas** object is a dataframe. A DataFrame is a 2-dimensional data structure that can store data of different types (including characters, integers, floating point values, categorical data, even lists) in columns. 

![](https://pandas.pydata.org/docs/_images/01_table_dataframe.svg)

We'll now import some useful packages, set a random number seed (some examples use randomly generated data), set up some nice plot settings, and load a dataset.

We'll look at a dataframe of the *penguins* dataset. To show just the first 5 rows, I'll be using the `head()` method (there's also a `tail()` method).


In [ ]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# Set seed for reproducibility
np.random.seed(10)
# Plot settings
plt.style.use(
    "https://github.com/aeturrell/coding-for-economists/raw/main/plot_style.txt"
)

df = sns.load_dataset("penguins")
df.head()

What just happened? We loaded a pandas dataframe called `df` and showed its contents. You can see the column names in bold, and the index on the left hand side. Just to double check it *is* a pandas dataframe, we can call type on this.

In [ ]:
type(df)

And if we want a bit more information about what we imported (including the datatypes of the columns):

In [ ]:
df.info()

Remember that everything in Python is an object, and our dataframe is no exception. Each dataframe is made up of a set of series that, in a dataframe, become columns: but you can turn a single series into a dataframe too. 

![](https://pandas.pydata.org/docs/_images/01_table_series.svg)

Let's see a couple of ways of creating some series from raw data:

In [ ]:
# From a list:
s1 = pd.Series([1.0, 6.0, 19.0, 2.0])
print(s1)
print("\n")
# From a dictionary
population_dict = {
    "California": 38332521,
    "Texas": 26448193,
    "New York": 19651127,
    "Florida": 19552860,
    "Illinois": 12882135,
}
s2 = pd.Series(population_dict)
print(s2)

Note that in each case there is no column name (because this is a series, not a dataframe), and there *is* an index. The index is automatically created if we don't specify it; in the third example, by passing a dictionary we implicitly asked for the index to be the locations we supplied. 

If you ever need to get the data 'out' of a series or dataframe, you can just call the `values` method on the object:

In [ ]:
s2.values

If you ever want to turn a series into a dataframe, just called `pd.DataFrame(series)` on it. Note that while series have an index and an object name (eg `s2` above), they don't have any column labels because they only have one column.

Now let's try creating our own dataframe with more than one column of data using a *dictionary*:

In [ ]:
df = pd.DataFrame(
    {
        "A": 1.0,
        "B": pd.Series(1, index=list(range(4)), dtype="float32"),
        "C": [3] * 4,
        "D": pd.Categorical(["test", "train", "test", "train"]),
        "E": "foo",
    }
)
df

Remember, curly brackets in the format `{key: value}` denote a dictionary. In the above example, the `pd.DataFrame()` function understands that any single value entries in the dictionary that is passed, such as `{'A': 1.}`, should be repeated as many times as are needed to match the longest series in the dictionary (4 in the above example).

Another way to create dataframes is to pass a bunch of series (note that `index`, `columns`, and `dtype` are optional--you can just specify the data):

In [ ]:
df = pd.DataFrame(
    data=np.reshape(range(36), (6, 6)),
    index=["a", "b", "c", "d", "e", "f"],
    columns=["col" + str(i) for i in range(6)],
    dtype=float,
)
df

Note that `reshape` takes an input and puts it into a given shape (a 6 by 6 matrix in the above example).

### Values, columns, and index

You'll have seen that there are three different things that make up a dataframe: the values that are in the cells, the column names, and the index. The column and index can take on values that are the same as the values in a dataframe do; string, int, float, datetime, and more. It's pretty obvious what role the columns play: they keep track of the name of different sets of values. But for people who may have seen other dataframe-like libraries, the role played by the index may be less familiar. The easiest way to think about a **pandas** index is that it does for row values what the columns do for columnar values: it's a way of keeping track of what individual roles are and it *doesn't* get used for calculations (just as summing a column ignores the name of the row).

Here's an example to show this. Let's first create a simple dataframe:

In [ ]:
df = pd.DataFrame(
    data={
        "col0": [0, 0, 0, 0],
        "col1": [0, 0, 0, 0],
        "col2": [0, 0, 0, 0],
        "col3": ["a", "b", "b", "a"],
        "col4": ["alpha", "gamma", "gamma", "gamma"],
    },
    index=["row" + str(i) for i in range(4)],
)
df.head()

If we add one to the integer columns in the dataframe, this is what we get (note we're not saving the result):

In [ ]:
df[["col0", "col1", "col2"]] + 1

Now let's use `col0` as our index instead of the original labels we created and add one to the remaining numeric columns:

In [ ]:
df = df.set_index("col0")
df[["col1", "col2"]] = df[["col1", "col2"]] + 1
df.head()

What was a column name has become an index name (which you can change with `df.index.name='newname'`) and, when we do add one, it isn't applied to the index. Even though their datatype is `int`, for integer, the index entries are now acting as a label for each row--not as values in the dataframe.

An index can be useful for keeping track of what's going on, and it's particularly convenient for some datetime operations.

Whenever you use `groupby` (and some other operations), the columns you use to perform the operation are set as the index of the returned dataframe (you can have multiple index columns). To get back those back to being columns, use the `reset_index()` method like so:

In [ ]:
df.groupby(["col3", "col4"]).sum()

In [ ]:
df.groupby(["col3", "col4"]).sum().reset_index()

## Datatypes

Pandas has some built-in datatypes (some are the basic Python datatypes) that will make your life a *lot* easier if you work with them. Why bother specifying datatypes? Languages like Python let you get away with having pretty much anything in your columns. But this can be a problem: sometimes you'll end up mixing integers, strings, the generic 'object' datatype, and more by mistake. By ensuring that columns conform to a datatype, you can save yourself from some of the trials that come with these mixed datatypes. Some of the most important datatypes for dataframe are string, float, categorical, datetime, int, and boolean. 

Typically, you'll read in a dataset where the dataypes of the columns are a mess. One of the first things you'll want to do is sort these out. Here's an example dataset showing how to set the datatypes:

In [ ]:
data = [
    ["string1", "string2"],
    [1.2, 3.4],
    ["type_a", "type_b"],
    ["01-01-1999", "01-01-2000"],
    [1, 2],
    [0, 1],
]
columns = [
    "string_col",
    "double_col",
    "category_col",
    "datetime_col",
    "integer_col",
    "bool_col",
]

df = pd.DataFrame(data=np.array(data).T, columns=columns)
df.info()

Note that the data type for all of these columns is the generic 'Object' (you can see this from the `Dtype` column that is printed when you use `df.info()`). Let's fix that:


In [ ]:
df = df.assign(
    string_col=df["string_col"].astype("string"),
    double_col=df["double_col"].astype("double"),
    category_col=df["category_col"].astype("category"),
    datetime_col=df["datetime_col"].astype("datetime64[ns]"),
    integer_col=df["integer_col"].astype("int"),
    bool_col=df["bool_col"].astype("bool"),
)
df.info()

If you're creating a series or dataframe from scratch, here's how to start off with these datatypes:


In [ ]:
str_s = pd.Series(["string1", "string2"], dtype="string")
float_s = pd.Series([1.2, 3.4], dtype=float)
cat_s = pd.Series(["type_a", "type_b"], dtype="category")
date_s = pd.Series(["01-01-1999", "01-01-2000"], dtype="datetime64[ns]")
int_s = pd.Series([1, 2], dtype=int)
bool_s = pd.Series([True, False], dtype=bool)

df = pd.concat([str_s, float_s, cat_s, date_s, int_s, bool_s], axis=1)
df.info()

### Categorical variables

Categorical variables can be especially useful and there are a couple of convenience functions that allow you to create them from other types of columns. Cut splits input data into a given number of (evenly spaced) bins that you can optionally give names to via the `labels=` keyword. The default behaviour is for the order of the labels to matter so that, in the below example, the bins will be evenly spaced from the smallest value to the largest, with the smallest receiving the label `bad`.



In [ ]:
pd.cut([1, 7, 5, 4, 6, 3], 3, labels=["bad", "medium", "good"])

We can also pass the bins directly:

In [ ]:
pd.cut([1, 7, 5, 4, 6, 3], bins=[-5, 0, 5, 10])

Another useful function is qcut, which provides a categorical breakdown according to a given number of quantiles (eg 4 produces quartiles):

In [ ]:
pd.qcut(range(1, 10), 4)

## Accessing and slicing

Now you know how to put data in a dataframe, how do you access the bits of it you need? There are various ways. If you want to access an entire column, the syntax is very simple; `df['columname']` (you can also use `df.columname`).

![](https://pandas.pydata.org/docs/_images/03_subset_columns.svg)




In [ ]:
df = pd.DataFrame(
    data=np.reshape(range(36), (6, 6)),
    index=["a", "b", "c", "d", "e", "f"],
    columns=["col" + str(i) for i in range(6)],
    dtype=float,
)
df["col1"]

To access a particular row, it's `df.loc['rowname']` or `df.loc[['rowname1', 'rowname1']]`.

![](https://pandas.pydata.org/docs/_images/03_subset_rows.svg)


In [ ]:
df.loc[["a", "b"]]

As well as the `.loc` method, there is the `.iloc` method that accesses rows or columns based on their position, for example `df.iloc[i, :]` for the ith row and `df.iloc[:, j]` for the jth column (but remember the numbers start from zero).

To access an individual value from within the dataframe, we have two options: pass an index value and a column name to `.loc[rowname, columnname]` or retrieve the value by using its position using `.iloc[row, column]`: 

In [ ]:
# Using .loc
print(df.loc["b", "col1"])
# Using .iloc
print(df.iloc[1, 0])

With all of these different ways to access values in dataframes, it can get confusing. These are the different ways to get the first column of a dataframe (when that first column is called `column` and the dataframe is `df`):

- `df.column`
- `df["column"]`
- `df.loc[:, "column"]`
- `df.iloc[:, 0]`

The ways to access rows are similar (here assuming the first row is called `row`):

- `df.loc["row", :]`
- `df.iloc[0, :]`

And to access the first value (ie the value in first row, first column):

- `df.column[0]`
- `df["column"][0]`
- `df.iloc[0, 0]`
- `df.loc["row", "column"]`

In the above examples, square brackets are instructions about *where* to grab bits from the dataframe. They are a bit like an address system for values within a dataframe.

Square brackets *also* denote lists though. So if you want to select *multiple* columns or rows, you might see syntax like this:

`df.loc[["row0", "row1"], ["column0", "column2"]]`

which picks out two rows and two columns via the lists `["row0", "row1"]` and `["column0", "column2"]`. Because there are lists alongside the usual system of selecting values, there are two sets of square brackets.

So often what we really want is a subset of values (as opposed to *all* values or just *one* value). This is where *slicing* comes in. If you've looked at the Basics of Coding chapter, you'll know a bit about slicing and indexing already, but we'll cover the basics here too.

The syntax for slicing is similar to what we've seen already: there are two methods `.loc` to access items by name, and `.iloc` to access them by position. The syntax for the former is `df.loc[start:stop:step, start:stop:step]`, where the first position is index name and the second is column name (and the same applies for numbers and `df.iloc`). Let's see an example using the storms dataset, and do some cuts.

In [ ]:
df.loc["a":"f":2, "col1":"col3"]

As you can see, slicing even works on names! By asking for rows `'a':'f':2`, we get every other row from 'a' to 'f' (inclusive). Likewise, for columns, we asked for every column between `col1` and `col3` (inclusive). `iloc` works in a very similar way.

In [ ]:
df.iloc[1:, :-1]

In this case, we asked for everything from row 1 onwards, and everything up to (but excluding) the last column.

It's not just strings and positions that can be sliced though, here's an example using *dates* (pandas support for dates is truly excellent):

In [ ]:
index = pd.date_range("1/1/2000", periods=12, freq="Q")
df = pd.DataFrame(np.random.randint(0, 10, (12, 5)), index=index, columns=list("ABCDE"))
df

Now let's do some slicing!

In [ ]:
df.loc["2000-01-01":"2002-01-01", :]

Two important points to note here: first, pandas doesn't mind that we supplied a date that didn't actually exist in the index. It worked out that by '2000-01-01' we meant a datetime and compared the values of the index to that datetime in order to decide what rows to return from the dataframe. The second thing to notice is the use of `:` for the column names; this explicitly says 'give me all the columns'.

## Operations on dataframes

Columns in dataframes can undergo all the usual arithmetic operations you'd expect of addition, multiplication, division, and so on. If the underlying datatypes of two columns have a group operation, then the dataframe columns will use that. 

![](https://github.com/pandas-dev/pandas/raw/059c8bac51e47d6eaaa3e36d6a293a22312925e6/doc/source/_static/schemas/05_newcolumn_1.svg)

The results of these manipulations can just be saved as a new series, eg, `new_series = df['A'] + df['B']` or created as a new column of the dataframe:

In [ ]:
df["new_col"] = df["A"] * (df["B"] ** 2) + 1
df

Boolean variables and strings have group operations (eg concatenation is via `+` with strings), and so work well with column operations too:

In [ ]:
df = pd.DataFrame(
    {"a": [1, 0, 1], "b": [0, 1, 1], "c": [0, 1, 1], "d": [1, 1, 0]}, dtype=bool
)
print(df)
print("\n a and c:\n")
print(df["a"] & df["c"])
print("\n b or d:\n")
print(df["b"] | df["d"])

More complex operations on whole dataframes are supported, but if you're doing very heavy lifting you might want to just switch to using numpy arrays (**numpy** is basically Matlab in Python). As examples though, you can transpose and exponentiate easily:

In [ ]:
df = pd.DataFrame(np.random.randint(0, 5, (3, 5)), columns=list("ABCDE"))
print("\n Dataframe:")
print(df)
print("\n Exponentiation:")
print(np.exp(df))
print("\n Transpose:")
print(df.T)

## Aggregation

**pandas** has built-in aggregation functions such as

| Aggregation      | Description |
| ----------- | ----------- |
| `count()`      | Number of items       |
| `first()`, `last()` | 	First and last item |
| `mean()`, `median()` |	Mean and median |
| `min()`, `max()` |	Minimum and maximum |
| `std()`, `var()` |	Standard deviation and variance |
| `mad()` |	Mean absolute deviation |
| `prod()` |	Product of all items |
| `sum()`	| Sum of all items |
| `value_counts()` | Counts of unique values |

these can applied to all entries in a dataframe, or optionally to rows or columns using `axis=0` or `axis=1` respectively.


In [ ]:
df.sum(axis=0)

## Split, apply, and combine

Splitting a dataset, applying a function, and combining the results are three key operations that we'll want to use again and again. Splitting means differentiating between rows or columns of data based on some conditions, for instance different categories or different values. Applying means applying a function, for example finding the mean or sum. Combine means putting the results of these operations back into the dataframe, or into a variable. The figure gives an example

![](https://jakevdp.github.io/PythonDataScienceHandbook/figures/03.08-split-apply-combine.png)

Note that the 'combine' part doesn't always have to result in a new dataframe; it could create new columns in an existing dataframe.

Let's first see a really simple example of splitting a dataset into groups and finding the mean across those groups using the *penguins* dataset. We'll group the data by island and look at the means. 

In [ ]:
df = sns.load_dataset("penguins")
df.groupby("island").mean()

The aggregations from the previous part all work on grouped data. An example is `df['body_mass_g'].groupby('island').std()` for the standard deviation of body mass by island.

You can also pass other functions via the `agg` method (short for aggregation). Here we pass two numpy functions:


In [ ]:
df.groupby("species").agg([np.mean, np.std])

Multiple aggregations can also be performed at once on the entire dataframe by using a dictionary to map columns into functions. You can also group by as many variables as you like by passing the groupby method a list of variables. Here's an example that combines both of these features:


In [ ]:
df.groupby(["species", "island"]).agg({"body_mass_g": "sum", "bill_length_mm": "mean"})

Sometimes, inheriting the column names becomes problematic. There's a slightly fussy syntax to help with that:


In [ ]:
df.groupby(["species", "island"]).agg(
    count_bill=("bill_length_mm", "count"),
    mean_bill=("bill_length_mm", "mean"),
    std_flipper=("flipper_length_mm", np.std),
)

Finally, you should know about the `apply` method, which takes a function and applies it to a given axis (`axis=0` for index, `axis=1` for columns) or column. The simple example below shows how it works, though in practice you'd just use `df['body_mass_kg'] = df['body_mass_g]/1e3` to do this.

In [ ]:
def g_to_kg(mass_in_g):
    return mass_in_g / 1e3


df["mass_in_kg"] = df["body_mass_g"].apply(g_to_kg)
df.head()

## Filter, transform, apply, and assign

### Filter

Filtering does exactly what it sounds like, but it can make use of group-by commands. In the example below, all but one species is filtered out.

In the example below, `filter` passes a grouped version of the dataframe into the `filter_func` we've defined (imagine that a dataframe is passed for each group). Because the passed variable is a dataframe, and variable `x` is defined in the function, the `x` within `filter_func` body behaves like our dataframe--including having the same columns.

In [ ]:
def filter_func(x):
    return x["bill_length_mm"].mean() > 48


df.groupby("species").filter(filter_func).head()

### Transform

Transforms return a transformed version of the data that has the same shape as the input. This is useful when creating new columns that depend on some grouped data, for instance creating group-wise means. Here's an example using the datetime group to subtract a yearly mean. First let's create some synthetic data with some data, a datetime index, and some groups:

In [ ]:
index = pd.date_range("1/1/2000", periods=10, freq="Q")
data = np.random.randint(0, 10, (10, 2))
df = pd.DataFrame(data, index=index, columns=["values1", "values2"])
df["type"] = np.random.choice(["group" + str(i) for i in range(3)], 10)
df

Now we take the yearly means by type. `pd.Grouper(freq='A')` is an instruction to take the `A`nnual mean using the given datetime index. You can group on as many coloumns and/or index properties as you like: this example groups by a property of the datetime index and on the `type` column, but performs the computation on the `values1` column.

In [ ]:
df["v1_demean_yr_type"] = df.groupby([pd.Grouper(freq="A"), "type"])[
    "values1"
].transform(lambda x: x - x.mean())
df

You'll have seen there's a `lambda` keyword here. Lambda (or anonymous) functions have a rich history in mathematics, and were used by scientists such as Church and Turing to create proofs about what is computable *before electronic computers existed*. They can be used to define compact functions:

In [ ]:
multiply_plus_one = lambda x, y: x * y + 1
multiply_plus_one(3, 4)

### Apply


Both regular functions and lambda functions can be used with the more general apply method, which takes a function and applies it to a given axis (`axis=0` for index, `axis=1` for columns):

In [ ]:
df["val1_times_val2"] = df.apply(lambda row: row["values1"] * row["values2"], axis=1)
df

Of course, the much easier way to do this very common operation is `df['val1_times_val2'] = df['values1']*df['values2']`, but there are times when you need to run more complex functions element-wise and, for those, `apply` is really useful.

### Assign

Assign is a method that allows you to return a new object with all the original columns in addition to new ones. Existing columns that are re-assigned will be overwritten. This is *really* useful when you want to perform a bunch of operations together in a concise way and keep the original columns. For instance, to demean the 'values1' column by year-type and to recompute the 'val1_times_val2' column using the newly demeaned 'values1' column:

In [ ]:
df.assign(
    values1=(
        df.groupby([pd.Grouper(freq="A"), "type"])["values1"].transform(
            lambda x: x - x.mean()
        )
    ),
    val1_times_val2=lambda x: x["values1"] * x["values2"],
)

## Time series, resampling, and rolling windows

The support for time series and the datetime type is excellent in pandas. It is very easy to manipulate datetimes. The [relevant part](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html) of the documentation has more info; here we'll just see a couple of the most important bits. First, let's create some synthetic data to work with:

In [ ]:
def recursive_ts(n, x=0.05, beta=0.6, alpha=0.2):
    shock = np.random.normal(loc=0, scale=0.6)
    if n == 0:
        return beta * x + alpha + shock
    else:
        return beta * recursive_ts(n - 1, x=x) + alpha + shock


t_series = np.cumsum([recursive_ts(n) for n in range(12)])
index = pd.date_range("1/1/2000", periods=12, freq="M")
df = pd.DataFrame(t_series, index=index, columns=["values"])
df.loc["2000-08-31", "values"] = np.nan
df

Now let's imagine that there are a number of issues with this time series. First, it's been recorded wrong: it actually refers to the start of the next month, not the end of the previous as recorded; second, there's a missing number we want to interpolate; third, we want to take the difference of it to get to something stationary; fourth, we'd like to add a lagged column. We can do all of those things!


In [ ]:
# Change freq to next month start
df.index += pd.tseries.offsets.DateOffset(days=1)

df["values"] = df["values"].interpolate(method="time")
df["diff_values"] = df["values"].diff(1)
df["lag_diff_values"] = df["diff_values"].shift(1)
df

Two other useful time series functions to be aware of are `resample` and `rolling`. `resample` can upsample or downsample time series. Downsampling is by aggregation, eg `df['values].resample('Q').mean()` to downsample to quarterly ('Q') frequency by taking the mean within each quarter. Upsampling involves a choice about how to fill in the missing values; examples of options are `bfill` (backfill) and `ffill` (forwards fill).

Rolling is for taking rolling aggregations, as you'd expect; for example, the 3-month rolling mean of our first difference time series:

In [ ]:
df["diff_values"].rolling(3).mean()


## Method chaining

Sometimes, rather than splitting operations out into multiple lines, it can be more concise and clear to chain methods together. A typical time you might do this is when reading in a dataset and perfoming all of the initial cleaning. Tom Augsperger has a [great tutorial](https://tomaugspurger.github.io/method-chaining) on this, which I've reproduced parts of here. For more info on the `pipe` function used below, check out these short [video tutorials](https://calmcode.io/pandas-pipe/introduction.html).

To chain methods together, both the input and output must be a pandas dataframe. Many functions already do input and output these, for example the `df.rename(columns={'old_col': 'new_col'})` takes in `df` and outputs a dataframe with one column name changed.

But occassionally, we'll want to use a function that we've defined (rather than an already existing one). For that, we need the `pipe` method; it 'pipes' the result of one operation to the next operation. When objects are being passed through multiple functions, this can be much clearer. Compare, for example,

```python
f(g(h(df), g_arg=a), f_arg=b)
```

that is, dataframe `df` is being passed to function `h`, and the results of that are being passed to a function `g` that needs a key word argument `g_arg`, and the results of *that* are being passed to a function `f` that needs keyword argument `f_arg`. The nested structure is barely readable. Compare this with

```python
(df.pipe(h)
   .pipe(g, g_arg=a)
   .pipe(f, f_arg=b)
)  
```

Let's see a method chain in action on a real dataset so you get a feel for it. We'll use 1,000 rows of flight data from BTS (a popular online dataset for demos of data cleaning!). TODO use github path. (For further info on method chaining in Python, [see these videos](https://calmcode.io/method-chains/introduction.html)--but be aware they assume advanced knowledge of the language.)

In [ ]:
df = pd.read_csv(
    "https://github.com/aeturrell/coding-for-economists/raw/main/data/flights1kBTS.csv",
    index_col=0,
)
df.head()

We'll try and do a number of operations in one go: putting column titles in lower case, discarding useless columns, creating precise depature and arrival times, turning some of the variables into categoricals, creating a demeaned delay time, and creating a new categorical column for distances according to quantiles that will be called 'near', 'less near', 'far', and 'furthest'. Some of these operations require a separate function, so we first define those. When we do the cleaning, we'll pipe our dataframe to those functions (optionally passing any arguments).

In [ ]:
def extract_city_name(df):
    """
    Chicago, IL -> Chicago for origin_city_name and dest_city_name
    """
    cols = ["origin_city_name", "dest_city_name"]
    city = df[cols].apply(lambda x: x.str.extract("(.*), \w{2}", expand=False))
    df = df.copy()
    df[["origin_city_name", "dest_city_name"]] = city
    return df


def time_to_datetime(df, columns):
    """
    Combine all time items into datetimes.

    2014-01-01,0914 -> 2014-01-01 09:14:00
    """
    df = df.copy()

    def converter(col):
        timepart = (
            col.astype(str)
            .str.replace("\.0$", "")  # NaNs force float dtype
            .str.pad(4, fillchar="0")
        )
        return pd.to_datetime(
            df["fl_date"]
            + " "
            + timepart.str.slice(0, 2)
            + ":"
            + timepart.str.slice(2, 4),
            errors="coerce",
        )

    df[columns] = df[columns].apply(converter)
    return df


df = (
    df.drop([x for x in df.columns if "Unnamed" in x], axis=1)
    .rename(columns=str.lower)
    .pipe(extract_city_name)
    .pipe(time_to_datetime, ["dep_time", "arr_time"])
    .assign(
        fl_date=lambda x: pd.to_datetime(x["fl_date"]),
        dest=lambda x: pd.Categorical(x["dest"]),
        origin=lambda x: pd.Categorical(x["origin"]),
        tail_num=lambda x: pd.Categorical(x["tail_num"]),
        arr_delay=lambda x: pd.to_numeric(x["arr_delay"]),
        op_unique_carrier=lambda x: pd.Categorical(x["op_unique_carrier"]),
        arr_delay_demean=lambda x: x["arr_delay"] - x["arr_delay"].mean(),
        distance_group=lambda x: (
            pd.qcut(x["distance"], 4, labels=["near", "less near", "far", "furthest"])
        ),
    )
)
df.head()

### Pyjanitor and more extensive method chaining

Although there's enough support for functional style method chaining to get by, you might find that doing this or that operation isn't always as efficient or as concise as you'd like. Let's look at two examples where this bites and what a wonderful little extension to **pandas** called [**Pyjanitor**](https://pyjanitor.readthedocs.io) does about it.

First, you'll have seen that the syntax for assigning a new column as part of a method chain is relatively fussy, using as it does the `new_column=lambda x: func(x['old_column])` syntax. If you already have a dataframe named `df`, **pyjanitor** gives you the option to create new columns like so (using an example from the documentation):


```python

df = (
    df.add_columns(
        prop_late_departures=df.num_departing_late / df.total_num_trips,
        prop_late_arrivals=df.num_arriving_late / df.total_num_trips
    )
)
```

What's great about this is not only that it's cleaner to read, but that all you need do to get the functionality is to import the pyjanitor library and the extra functions will appear magically as options to use on the usual pandas dataframes.

Pyjanitor isn't just about method chaining--it does a lot of other things too, like introducing a bunch of convenience commands with easily understandable verbs behind them. (Though do note it has a hefty download size.)

## Reshaping data

The main options for reshaping data are `pivot`, `melt`, `stack`, `unstack`, `pivot_table`, `get_dummies`, `cross_tab`, and `explode`. We’ll look at some of these here.



### Pivoting data from tidy to, err, untidy

At the start of this chapter, I said you should use tidy data--one row per observation, one column per variable--whenever you can. But there are times when you will want to take your lovingly prepared tidy data and pivot it into a wider format. `pivot` and `pivot_table` help you to do that.

![](https://pandas.pydata.org/docs/_images/reshaping_pivot.png)

This can be especially useful for time series data, where operations like `shift` or `diff` are typically applied assuming that an entry in one row follows (in time) from the one above. Here's an example:

In [ ]:
data = {
    "value": np.random.randn(20),
    "variable": ["A"] * 10 + ["B"] * 10,
    "date": (
        list(pd.date_range("1/1/2000", periods=10, freq="M"))
        + list(pd.date_range("1/1/2000", periods=10, freq="M"))
    ),
}
df = pd.DataFrame(data, columns=["date", "variable", "value"])
df.sample(5)

If we just run `shift` on this, it's going to shift variable B's and A's together. So we pivot to a wider format (and then we can shift safely).

In [ ]:
df.pivot(index="date", columns="variable", values="value").shift(1)


### Melt

`melt` can help you go from untidy to tidy data (from wide data to long data), and is a *really* good one to remember. Of course, I have to look at the documentation every single time myself, but I'm sure you'll do better.

![](https://pandas.pydata.org/docs/_images/reshaping_melt.png)

Here's an example of it in action:

In [ ]:
df = pd.DataFrame(
    {
        "first": ["John", "Mary"],
        "last": ["Doe", "Bo"],
        "height": [5.5, 6.0],
        "weight": [130, 150],
    }
)
print("\n Unmelted: ")
print(df)
print("\n Melted: ")
df.melt(id_vars=["first", "last"], var_name="quantity")

If you don't wan the headscratching of melt, there's also `wide_to_long`, which is really useful for typical data cleaning cases where you have data like this:

In [ ]:
df = pd.DataFrame(
    {
        "A1970": {0: "a", 1: "b", 2: "c"},
        "A1980": {0: "d", 1: "e", 2: "f"},
        "B1970": {0: 2.5, 1: 1.2, 2: 0.7},
        "B1980": {0: 3.2, 1: 1.3, 2: 0.1},
        "X": dict(zip(range(3), np.random.randn(3))),
        "id": dict(zip(range(3), range(3))),
    }
)
df

i.e. data where there are different variables and time periods across the columns. Wide to long is going to let us give info on what the stubnames are ('A', 'B'), the name of the variable that's always across columns (here, a year), any values (X here), and an id column.

In [ ]:
pd.wide_to_long(df, ["A", "B"], i="id", j="year")

### Stack and unstack

Stack, `stack()` is a shortcut for taking a single type of wide data variable from columns and turning it into a long form dataset, but with an extra index.

![](https://pandas.pydata.org/docs/_images/reshaping_stack.png)

Unstack, `unstack()` unsurprisingly does the same operation, but in reverse.

![](https://pandas.pydata.org/docs/_images/reshaping_unstack.png)

Let's define a multi-index dataframe to demonstrate this:

In [ ]:
tuples = list(
    zip(
        *[
            ["bar", "bar", "baz", "baz", "foo", "foo", "qux", "qux"],
            ["one", "two", "one", "two", "one", "two", "one", "two"],
        ]
    )
)
index = pd.MultiIndex.from_tuples(tuples, names=["first", "second"])
df = pd.DataFrame(np.random.randn(8, 2), index=index, columns=["A", "B"])
df

Let's stack this to create a tidy dataset:

In [ ]:
df = df.stack()
df

Now let's see unstack but, instead of unstacking the 'A', 'B' variables we began with, let's unstack the 'first' column by passing `level=0` (the default is to unstack the innermost index). This diagram shows what's going on:

![](https://pandas.pydata.org/docs/_images/reshaping_unstack_0.png)

And here's the code:

In [ ]:
df.unstack(level=0)

### Get dummies

This is a really useful reshape command for when you want (explicit) dummies in your dataframe. When running simple regressions, you can achieve the same effect by declaring the column only be included as a fixed effect, but there are some machine learning packages where converting to dummies may be easier.

Here's an example:

In [ ]:
df = pd.DataFrame(
    {"group_var": ["group1", "group2", "group3"], "B": ["c", "c", "b"], "C": [1, 2, 3]}
)
print(df)

pd.get_dummies(df, columns=["group_var"])


## Combining data

**pandas** has a really rich set of options for combining one or more dataframes. The two most important are concatenate and merge.

### Concatenate

If you have two or more dataframes with the same index or the same columns, you can glue them together into a single dataframe using `pd.concat`. 

![](https://pandas.pydata.org/docs/_images/08_concat_row.svg)

For the same columns, pass `axis=0` to glue the index together; for the same index, pass `axis=1` to glue the columns together. The concatenate function will typically be used on a list of dataframes.

If you want to track where the original data came from in the final dataframe, use the `keys` keyword.

Here's an example using data on two different states' populations that also makes uses of the `keys` option:

In [ ]:
base_url = "http://www.stata-press.com/data/r14/"
state_codes = ["ca", "il"]
end_url = "pop.dta"

# This grabs the two dataframes, one for each state
list_of_state_dfs = [pd.read_stata(base_url + state + end_url) for state in state_codes]

# Concatenate the list of dataframes
df = pd.concat(list_of_state_dfs, keys=state_codes, axis=0)
df

### Merge

There are so many options for merging dataframes using `pd.merge(left, right, on=..., how=...` that we won't be able to cover them all here. The most important features are: the two dataframes to be merged, what variables (aka keys) to merge on (and these can be indexes) via `on=`, and *how* to do the merge (eg left, right, outer, inner) via `how=`. This diagram shows an example of a merge using keys from the left-hand dataframe:

![](https://pandas.pydata.org/docs/_images/08_merge_left.svg)

The `how=` keyword works in the following ways:
- `how='left'` uses keys from the left dataframe only to merge.
- `how='right'` uses keys from the right dataframe only to merge.
- `how='inner'` uses keys that appear in both dataframes to merge.
- `how='outer'` uses the cartesian product of keys in both dataframes to merge on.

Let's see examples of some of these:

In [ ]:
left = pd.DataFrame(
    {
        "key1": ["K0", "K0", "K1", "K2"],
        "key2": ["K0", "K1", "K0", "K1"],
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
    }
)
right = pd.DataFrame(
    {
        "key1": ["K0", "K1", "K1", "K2"],
        "key2": ["K0", "K0", "K0", "K0"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    }
)
# Right merge
pd.merge(left, right, on=["key1", "key2"], how="right")

Note that the key combination of K2 and K0 did not exist in the left-hand dataframe, and so its entries in the final dataframe are NaNs. But it *does* have entries because we chose the keys from the right-hand dataframe.

What about an inner merge?

In [ ]:
pd.merge(left, right, on=["key1", "key2"], how="inner")

Now we see that the combination K2 and K0 are excluded because they didn't exist in the overlap of keys in both dataframes.

Finally, let's take a look at an outer merge that comes with some extra info via the `indicator` keyword:

In [ ]:
pd.merge(left, right, on=["key1", "key2"], how="outer", indicator=True)

Now we can see that the products of all key combinations are here. The `indicator=True` option has caused an extra column to be added, called '_merge', that tells us which dataframe the keys on that row came from.

For more on the options, see the **pandas** [merging documentation](https://pandas.pydata.org/docs/user_guide/merging.html#database-style-dataframe-or-named-series-joining-merging).

## (Advanced) alternatives to **pandas**

Feel free to skip this section if you're just interested in getting going as quickly as possible.

**pandas** isn't the only game in town, not by a long way--though it's by far the best supported and the most fully featured. But it's always good to have options--or, put another way, options have value! Other dataframe libraries may have a syntax that you prefer or provide a speed-up (perhaps in certain situations, for example when working with very large datasets).

If you're specifically interested in how different dataframe options perform on increasingly large datasets, take a look at the benchmarks [here](https://h2oai.github.io/db-benchmark/).

Here's a quick run-through of some alternatives to **pandas**:

### Datatable

[**datatable**](https://datatable.readthedocs.io/en/latest/) is another dataframe based way to do analysis, and it has quite a different syntax to **pandas** for data manipulation. **datatable** is very fast, not the fastest dataframe option out there, but it holds its own. Its other major advantage is that it is comfortable running on extremely large (think 50GB) datasets. Most other dataframe packages (including in other languages) cannot cope with this.

I am indebted to the very well-written datatable documentation for the rest of this demonstration.

In datatable, almost all operations are achieved via so-called 'square-bracket notation'. Operations with a (data) Frame are almost all expressed in the form

```python
DT[i, j, ...]
```

where `DT` is a datatable dataframe, `i` is the row selector, `j` is the column selector, and `...` is a space that could be filled by other commands (potentially several). However, `i` and `j` aren't just positional, as they might be in a **numpy** array or a list, they do a lot more than that. 

Datatable allows `i` to be anything that can conceivably be interpreted as a row selector: an integer to select a row, a slice, a range, a list of integers, a list of slices, an expression, a boolean-valued Frame, an integer-valued Frame, an integer **numpy** array, a generator, and more.

Likewise, `j` is also really versatile. It can be used to select a column by name or position, but it will also accept a list of columns, a slice, a list of booleans indicating which columns to pick, an expression, a list of expressions, and a dictionary of expressions. The j expression can even be a python type (such as int or dt.float32), selecting all columns matching that type!

To change an entry (entries), you can use `DT[i, j] = new_value`, and to remove it (or them), it's `del DT[i, j]`.

It's probably going to be easiest to see some examples, so let's do a quick run through of some functions with the storms dataset.

In [ ]:
from datatable import dt, f, by, g, join, sort, update, ifelse

DT = dt.fread("https://vincentarelbundock.github.io/Rdatasets/csv/dplyr/storms.csv")

DT.head(5)

Yes, that's right, you're seeing columns colour coded by data type! Otherwise, what you can see here is that the syntax so far looks quite similar but don't worry because it's about to get weird.

The square brackets accept expressions, i.e. functions. This is achieved through a special import, simply denoted `f`, that says do this thing to the current datatable. For example, to filter by a certain value in the 'status' column (to avoid overloading with data, I'll just use head to show the first few rows):

In [ ]:
DT[f.status == "tropical storm", :].head(4)

Let's get rid of some columns we're not using right now

In [ ]:
del DT[:, "year":"long"]
DT.head(4)

Now, to select only those columns that are strings:


In [ ]:
DT[:, str].head(4)

Or to select only those columns that contain 'diameter'

In [ ]:
DT[:, [x for x in DT.names if "diameter" in x]].tail(4)

To create a new column based on an existing one and just look at the last few columns:

In [ ]:
DT[:, update(atmospheres=f.pressure / 1013.0)]
DT[:, -4:].tail(4)

To sort all values by multiple columns:

In [ ]:
DT[:, :, sort("wind", "pressure", reverse=[True, True])].head(4)

And to perform group-by operations:

In [ ]:
DT[:, dt.mean(f.wind), by("status")]

And, finally, one that I always find of great practical use--the within-group transform, with the new column going back into the original data(frame/table). In **pandas**, this is achieved by `transform`, here we use `extend`. Let's demonstrate by showing the maximum wind speed by status group:

In [ ]:
DT[:, f[:].extend({"max_wind": dt.min(f.wind)}), by("status")].head(4)

### cuDF

[**cuDF**](https://github.com/rapidsai/cudf) is still under development and so doesn't yet have all the features of **pandas**. It's a Python GPU DataFrame library built on the blisteringly fast Apache Arrow columnar memory format. The parts of the library that have been implemented follow the same commands and structure as **pandas**, so it should be easy to use it should you need to.

**cuDF** is *the* fastest dataframe library out there in any of Python, Julia, Rust, or R with the caveats that:
- this only applies to in-memory datasets, ie datasets smaller than a few GB
- it doesn't yet do everything
- it's only currently available on the Linux operating system
- you need a GPU (and associated software) to use it!

### Polars

[**Polars**]() is almost as fast as **cuDF**, but is further down the development path. It also uses Apache Arrow as backend. It currently consists of an 'eager' (for datasets smaller than approximately a few GB) interface that's very similar to **pandas** and a 'lazy' interface (don't worry if you don't know what that means, it's a big data thing) that is somewhat similar to spark (a big data tool). **Polars** is built on the Rust language. It's particularly effective at merging datasets.

### Plydata

[Plydata](https://plydata.readthedocs.io/en/stable/index.html) uses the syntax of the **dplyr** package in the R statistical language (the package is a really important part of the data ecosystem in R). It's built around method chaining and everything being in a tidy format, which has pros and cons. Largely, the benefits are in readability and a smaller number of commands to cover the ground that you need to. The downsides are that it plays best with tidy data and when chained methods go wrong it can be more difficult to find out what the issue is.



### dfplyr

[dfplyr](https://github.com/kieferk/dfply) also follows the syntax of the **dplyr** package in R. *Note* that it does not appear to be under active development.

## Review

If you know:

- ✅ what tidy data are;
- ✅ how to create series and dataframes with different datatypes;
- ✅ how to access values in series and dataframes;
- ✅ how to perform operations on columns;
- ✅ how to chain methods;
- ✅ how to reshape data;
- ✅ how to combine different dataframes; and
- ✅ what some alternatives to **pandas** are

then you are well on your way to becoming a data analysis master!
